# [Pandas and HDF5](http://pandas.pydata.org/)

In [1]:
%matplotlib inline

In [2]:
import os
import shutil
import glob
import sqlite3 as sqlite
DATADIR = os.path.join(os.path.expanduser("~"),
                       "DATA", "Bioinf")
print(os.path.exists(DATADIR))

import pandas as pd
import seaborn as sns

sns.set()

True


## Reading/Writing Text Data with  Pandas

One of the beauties of Pandas is the ease of data input/output that it provides. It has the capability to read a variety of common data formats including
* Tabular text data
    * [``read_csv``](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html): read comma separated files
    * [``read_table``](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html): read tab separated files
        * These are both wrappers to the same function with different default values
    * [``read_excel``](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html)
* Databases
    * [``read_sql``](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql.html)
* Excel
    
* HDF5, a high performance file format for very large data
    * [``read_hdf``](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.read_hdf.html)



## What is HDF5?

> HDF5 is a data model, library, and file format for storing and managing data. It supports an unlimited variety of datatypes, and is designed for flexible and efficient I/O and for high volume and complex data. HDF5 is portable and is extensible, allowing applications to evolve in their use of HDF5. ([HDF5 Home Page](https://www.hdfgroup.org/HDF5/))

> HDF5 is a unique technology suite that makes possible the management of extremely large and complex data collections. ([HDF5 FAQ](https://www.hdfgroup.org/about/hdf_technologies.html))

The roots of HDF5 go back to the late 1980s at the National Center for Supercomputing Applications and was shortly thereafter adopted by NASA as the data form for its Earth Observing System ([Wikipedia](https://en.wikipedia.org/wiki/Hierarchical_Data_Format)).

## Why Use HDF5?

HDF5 offers both disk utilization and performance enhancements over naive file foramts, such as CSV files. We can illustrate this with gene expression data. In the ``/home/jovyan/DATA/Bioinf`` directory we have the PANCAN12 data file stored in two formats:

1. The original text file (``PANCAN12.IlluminaHiSeq_RNASeqV2.geneExp.tumor_whitelist``)
1. The data stored as an HDF5 file (``PANCAN12.IlluminaHiSeq_RNASeqV2.geneExp.tumor_whitelist.hdf5``

The HDF5 file is only about 2/3 the size of the original file. We will also see noticable improvements in how long it takes to read the data from disk.

```bash
-rw-r--r-- 1 jovyan staff 538029908 Jun  8 20:49 PANCAN12.IlluminaHiSeq_RNASeqV2.geneExp.tumor_whitelist.hdf5                      
-rwxr-xr-x 1 jovyan staff 839788724 Jun  8 20:50 PANCAN12.IlluminaHiSeq_RNASeqV2.geneExp.tumor_whitelist                           
```

## Simple Profiling

One simple way we can evaluate the performance of our programs is by using Python's ``time`` module to measure how  how long it takes for a command to execute. This is somewhat naive because our computers are multitasking and a program might take a longer or shorter amount of time because the computer was doing more or fewer competing tasks. But it is a reasonable starting approach.

Within the ``time`` module is a function [``time``](https://docs.python.org/3/library/time.html) that, on Unix systems, returns the number of elapsed seconds since January 1, 1970 ([see Unix epoch](https://en.wikipedia.org/wiki/Unix_time)).

We can save the Unix time before we start the command and compare that to the Unix time after our command execute. 

**Note:** it takes tens of seconds to read the file in.

In [3]:
import time
url_txt = \
    os.path.join(DATADIR,
                 "PANCAN12.IlluminaHiSeq_RNASeqV2.geneExp.tumor_whitelist")
start = time.time()
pd.read_table(url_txt)
time_table = time.time()-start
print("Elapsed time to read original file",time_table)
#data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


Elapsed time to read original file 42.98869180679321


### Now Measure Time to Read in HDF5

In [4]:
url_txt = os.path.join(DATADIR,"PANCAN12.IlluminaHiSeq_RNASeqV2.geneExp.tumor_whitelist.hdf5")
start = time.time()
pd.read_hdf(url_txt)
time_hdf5 = time.time()-start
print("Elapsed time to read hdf5 file",time_hdf5)

Elapsed time to read hdf5 file 1.1325137615203857


## Slicing

In [5]:
data[0:2]


NameError: name 'data' is not defined

## Pandas and HDF5

* HDF5 is a high performance binary data format written in C
* HDF5 facilitates a number of performance enhancements such as being able to access parts of the data without having to read into memory the whole dataset
* Python has too different packages provide an HDF5 interface
    * [h5py](http://www.h5py.org/)
    * [pytables](http://www.pytables.org/moin)
* Pandas uses pytable to interface with hdf5


In [6]:
!conda install pytables -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.7.5

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [7]:
url_hdf = os.path.join(DATADIR, "PANCAN12.IlluminaHiSeq_RNASeqV2.geneExp.tumor_whitelist.hdf5")
start = time.time()
data_hdf5 = pd.read_hdf(url_hdf)
time_hdf5 = time.time()-start
print("HDF5 %5.4f x faster than traditional read"%(time_table/time_hdf5))

HDF5 92.4795 x faster than traditional read


In [8]:
data_hdf5

,#probe,TCGA-02-0047,TCGA-02-0055,TCGA-02-2483,TCGA-02-2485,TCGA-02-2486,TCGA-04-1348,TCGA-04-1357,TCGA-04-1362,TCGA-04-1364,...,TCGA-HD-7831,TCGA-HD-7832,TCGA-HD-7917,TCGA-HN-A2NL,TCGA-HQ-A2OE,TCGA-IQ-7630,TCGA-IQ-7631,TCGA-IQ-7632,TCGA-J2-8192,TCGA-J2-8194
0,A1BG,125.006897,391.803802,271.852203,83.942902,108.256104,66.469498,65.566399,41.641201,187.036804,...,64.610901,15.324400,13.888900,132.303497,17.530600,51.436001,110.809097,86.582703,60.546200,136.266907
1,A1CF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.331000,0.000000,...,0.577500,0.000000,0.000000,0.000000,103.257103,0.000000,0.000000,0.000000,0.000000,0.000000
2,A2BP1,244.629501,137.351105,111.028999,257.142914,4.268300,0.268900,0.651000,4.302500,10.600600,...,46.195999,0.000000,0.420900,0.000000,0.231000,0.665800,3.879100,0.000000,2.252500,0.000000
3,A2LD1,55.072399,84.014000,34.537201,126.285698,190.530502,221.521896,141.282593,265.816101,94.247704,...,47.553101,97.793800,54.819000,143.108200,37.142502,91.892303,56.125000,58.433701,80.972298,91.273399
4,A2ML1,41.181400,1.681900,2.533000,211.428604,103.658501,7.528900,54.687500,5.626300,5.088300,...,236.177307,1167.309692,11734.006836,0.681300,3454.146484,434.792389,84.231796,5141.292480,43.172199,0.436100
5,A2M,34012.421875,42876.261719,21058.519531,7798.533203,40971.425781,5899.828125,9384.440430,3350.420654,1455.231567,...,13529.631836,1277.790161,2233.977295,5096.629395,2753.227051,1538.419312,12355.940430,1815.985840,26093.916016,23738.701172
6,A4GALT,36.264198,487.736511,86.965698,155.238098,43.902401,92.498001,298.177094,697.991882,147.984207,...,1530.821411,1234.303833,1365.319946,72.553902,1116.886108,602.583618,1796.574829,2748.630859,751.196594,211.513306
7,A4GNT,1.229300,0.560600,0.000000,0.476200,0.000000,0.537800,0.000000,0.000000,0.000000,...,0.577500,0.701500,0.000000,0.000000,0.231000,0.332900,1.108300,0.000000,2.252500,0.872200
8,AAA1,1.229300,0.000000,0.000000,0.000000,0.609800,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.340600,2.772000,0.000000,1.108300,0.000000,1.877100,0.000000
9,AAAS,466.517090,594.814270,1100.580444,1013.809509,701.829285,2186.071533,1073.567749,1140.814697,1607.473022,...,634.040710,763.942505,708.333313,1833.943604,1101.178101,1002.419495,1008.010315,825.848816,501.548614,497.601410


In [9]:
21000*3000*8


504000000